## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os,shutil
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.datasets import mnist
from keras.callbacks import ReduceLROnPlateau
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras import layers
from keras import models
from keras import optimizers
from keras_mixnets import MixNetLarge,MixNetSmall
import warnings
warnings.filterwarnings('ignore') 

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

x_train,x_val,y_train,y_val=train_test_split(x_train, y_train,test_size=0.25,random_state=13)

In [3]:
x_train.shape

(37500, 32, 32, 3)

In [4]:
LR_function=ReduceLROnPlateau(monitor='val_acc',
                             patience=3,
                             verbose=1,
                             factor=0.5,
                             min_lr=0.000001)

In [5]:
train_datagen = ImageDataGenerator(width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_datagen.fit(x_train)                                  

valid_datagen = ImageDataGenerator()
valid_datagen.fit(x_val)                             

In [6]:
model=MixNetLarge(input_shape=(32,32,3),classes=10, include_top=True,pooling='avg',)


model.summary()
model.compile(optimizer='Adam' , loss='categorical_crossentropy' , metrics=['accuracy'])

Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
groupconvolution_1 (GroupConvol (None, 16, 16, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 16, 16, 32)   128         groupconvolution_1[0][0]         
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 16, 16, 32)   0           batch_normalization_1[0][0]      
____

In [7]:
model.fit_generator(train_datagen.flow(x_train,y_train,batch_size=64),
                    steps_per_epoch = x_train.shape[0] / 64,
                    validation_data = (x_val,y_val),
                    validation_steps = x_val.shape[0] / 64,
                    epochs = 100,
                    callbacks=[LR_function])

Epoch 1/100
586/585 [==============================] - 80s 137ms/step - loss: 2.4395 - acc: 0.2054 - val_loss: 2.0482 - val_acc: 0.2751
Epoch 2/100
586/585 [==============================] - 68s 116ms/step - loss: 2.1859 - acc: 0.2570 - val_loss: 1.9182 - val_acc: 0.2904
Epoch 3/100
586/585 [==============================] - 66s 112ms/step - loss: 2.1526 - acc: 0.2629 - val_loss: 2.3354 - val_acc: 0.1614
Epoch 4/100
586/585 [==============================] - 63s 108ms/step - loss: 2.1854 - acc: 0.2543 - val_loss: 2.0966 - val_acc: 0.3178
Epoch 5/100
586/585 [==============================] - 66s 112ms/step - loss: 2.3116 - acc: 0.2170 - val_loss: 2.4550 - val_acc: 0.1738
Epoch 6/100
586/585 [==============================] - 66s 113ms/step - loss: 2.2941 - acc: 0.2025 - val_loss: 3.8765 - val_acc: 0.1894
Epoch 7/100
586/585 [==============================] - 64s 109ms/step - loss: 2.1461 - acc: 0.2520 - val_loss: 1.9786 - val_acc: 0.2945

Epoch 00007: ReduceLROnPlateau reducing learnin

In [8]:
loss,acc=model.evaluate(x_test,y_test)
print(f'The Accuracy on Test Data = {acc}')

10000/10000 [==============================] - 6s 627us/step
The Accuracy on Test Data = 0.4957
